In [1]:
import warnings
warnings.filterwarnings("ignore")

import datetime
import inspect
import io 
import json
import numpy as np
import pandas as pd
import requests 
from scipy.optimize import curve_fit


In [2]:
urlData = requests.get('https://corona.lmao.ninja/v2/historical/argentina').content

In [3]:
data = urlData.decode('utf-8')
data

'{"country":"Argentina","provinces":["mainland"],"timeline":{"cases":{"3/28/20":690,"3/29/20":745,"3/30/20":820,"3/31/20":1054,"4/1/20":1054,"4/2/20":1133,"4/3/20":1265,"4/4/20":1451,"4/5/20":1451,"4/6/20":1554,"4/7/20":1628,"4/8/20":1715,"4/9/20":1795,"4/10/20":1975,"4/11/20":1975,"4/12/20":2142,"4/13/20":2208,"4/14/20":2277,"4/15/20":2443,"4/16/20":2571,"4/17/20":2669,"4/18/20":2758,"4/19/20":2839,"4/20/20":2941,"4/21/20":3031,"4/22/20":3144,"4/23/20":3435,"4/24/20":3607,"4/25/20":3780,"4/26/20":3892},"deaths":{"3/28/20":18,"3/29/20":19,"3/30/20":23,"3/31/20":27,"4/1/20":28,"4/2/20":36,"4/3/20":39,"4/4/20":43,"4/5/20":44,"4/6/20":48,"4/7/20":56,"4/8/20":63,"4/9/20":72,"4/10/20":82,"4/11/20":83,"4/12/20":90,"4/13/20":97,"4/14/20":102,"4/15/20":111,"4/16/20":115,"4/17/20":123,"4/18/20":129,"4/19/20":132,"4/20/20":136,"4/21/20":147,"4/22/20":152,"4/23/20":165,"4/24/20":176,"4/25/20":185,"4/26/20":192},"recovered":{"3/28/20":72,"3/29/20":72,"3/30/20":228,"3/31/20":240,"4/1/20":248,"4/2/2

In [4]:
data_load = json.loads(data)
data_load

{'country': 'Argentina',
 'provinces': ['mainland'],
 'timeline': {'cases': {'3/28/20': 690,
   '3/29/20': 745,
   '3/30/20': 820,
   '3/31/20': 1054,
   '4/1/20': 1054,
   '4/2/20': 1133,
   '4/3/20': 1265,
   '4/4/20': 1451,
   '4/5/20': 1451,
   '4/6/20': 1554,
   '4/7/20': 1628,
   '4/8/20': 1715,
   '4/9/20': 1795,
   '4/10/20': 1975,
   '4/11/20': 1975,
   '4/12/20': 2142,
   '4/13/20': 2208,
   '4/14/20': 2277,
   '4/15/20': 2443,
   '4/16/20': 2571,
   '4/17/20': 2669,
   '4/18/20': 2758,
   '4/19/20': 2839,
   '4/20/20': 2941,
   '4/21/20': 3031,
   '4/22/20': 3144,
   '4/23/20': 3435,
   '4/24/20': 3607,
   '4/25/20': 3780,
   '4/26/20': 3892},
  'deaths': {'3/28/20': 18,
   '3/29/20': 19,
   '3/30/20': 23,
   '3/31/20': 27,
   '4/1/20': 28,
   '4/2/20': 36,
   '4/3/20': 39,
   '4/4/20': 43,
   '4/5/20': 44,
   '4/6/20': 48,
   '4/7/20': 56,
   '4/8/20': 63,
   '4/9/20': 72,
   '4/10/20': 82,
   '4/11/20': 83,
   '4/12/20': 90,
   '4/13/20': 97,
   '4/14/20': 102,
   '4/15/20

In [5]:
df_timeline = pd.json_normalize(data_load['timeline']['cases']).T
df_deaths = pd.json_normalize(data_load['timeline']['deaths']).T
df_recovered = pd.json_normalize(data_load['timeline']['recovered']).T

In [6]:
df_timeline.columns = ['confirmed']
df_deaths.columns = ['deaths']
df_recovered.columns = ['recovered']

In [7]:
data_argentina = pd.concat([df_timeline, df_deaths, df_recovered], axis=1, sort=False)
data_argentina

,confirmed,deaths,recovered
3/28/20,690,18,72
3/29/20,745,19,72
3/30/20,820,23,228
3/31/20,1054,27,240
4/1/20,1054,28,248
4/2/20,1133,36,256
4/3/20,1265,39,266
4/4/20,1451,43,279
4/5/20,1451,44,280
4/6/20,1554,48,325


In [8]:
data_argentina.index=pd.to_datetime(data_argentina.index.astype(str), format='%m/%d/%y')

In [9]:
data_argentina['num_id'] = np.arange(len(data_argentina) + 1)[1:]

In [10]:
data_argentina

,confirmed,deaths,recovered,num_id
2020-03-28,690,18,72,1
2020-03-29,745,19,72,2
2020-03-30,820,23,228,3
2020-03-31,1054,27,240,4
2020-04-01,1054,28,248,5
2020-04-02,1133,36,256,6
2020-04-03,1265,39,266,7
2020-04-04,1451,43,279,8
2020-04-05,1451,44,280,9
2020-04-06,1554,48,325,10


In [11]:
predicted_days = 7
firstPredictionDay = data_argentina.index[-1] + datetime.timedelta(days = 1)

In [12]:
class Models:    
    def cuadratic_model(x, a, b, c):
        return (a*x**2 + b*x +c)
    
    def cubic_model(x, a, b, c, d):
        return (a*x**3 + b*x**2 +c*x + d)
    
    def fourth_pow_model(x, a, b, c, d, e):
        return (a*x**4 + b*x**3 +c*x**2 + d*x + e)
    
    def fiveth_pow_model(x, a, b, c, d, e, f):
        return (a*x**5 + b*x**4 +c*x**3 + d*x**2 + e*x + f)
    
    def sixh_pow_model(x, a, b, c, d, e, f, g):
        return (a*x**6 + b*x**5 +c*x**4 + d*x**3 + e*x**2 + f*x + g)
    
    def exp_model(x, a, b):
        return a * np.exp(b * x)
    
    def complex_cuadratic_model(x, a, b, c, n2):
        return (a*(x - n2)**2 + b*x + c)
    
    def complex_cubic_model(x, a, b, c, d, n3, n2):
        return (a*(x - n3)**3 + b*(x - n2)**2 +c*x + d)

In [13]:
def formatPrediction(firstPredictionDay, day, prediction):
    formatedDate = str(firstPredictionDay + datetime.timedelta(days = day))[0:10]
    formatedPrediction = str(int(prediction))
    return "|" + formatedDate + " | " + formatedPrediction + "|"

def printPrediction(model, popt, x, y):
    firstPredictionDayId = x.max() + 1
    print("--- Predictions ---")
    for day in range (0, predicted_days):
        predictionId = firstPredictionDayId + day
        predictionValue = model(predictionId, *popt)
        print(formatPrediction(firstPredictionDay, day, predictionValue))
    print("-------------------")

In [14]:
def printFuncionParameters(model, popt):
        print('\n', model.__name__)
        args = inspect.getfullargspec(model).args[1:]
        for arg, value in zip(args, popt):
            print( arg, " = ", value)

In [15]:
def predictData(model, x, y):
    popt, _ = curve_fit(model, x, y)
    printPrediction(model, popt, x, y)
    printFuncionParameters(model, popt)
    print()

In [16]:
def isModel(methodName):
    return not(methodName.startswith('__'))

for methodName in dir(Models):
    if(isModel(methodName)):
        print(methodName)
        predictData(getattr(Models, methodName), data_argentina.num_id, data_argentina.confirmed)

complex_cuadratic_model
--- Predictions ---
|2020-04-27 | 3939|
|2020-04-28 | 4078|
|2020-04-29 | 4218|
|2020-04-30 | 4360|
|2020-05-01 | 4505|
|2020-05-02 | 4651|
|2020-05-03 | 4799|
-------------------

 complex_cuadratic_model
a  =  1.0195952457721522
b  =  46.47123729415819
c  =  481.12547463185547
n2  =  -13.49052077577091

complex_cubic_model
--- Predictions ---
|2020-04-27 | 4082|
|2020-04-28 | 4276|
|2020-04-29 | 4480|
|2020-04-30 | 4695|
|2020-05-01 | 4923|
|2020-05-02 | 5162|
|2020-05-03 | 5414|
-------------------

 complex_cubic_model
a  =  0.0872859961902701
b  =  -2.3012391746816223
c  =  210.34464868524037
d  =  1353.440516656785
n3  =  2.8181857883074364
n2  =  -18.9648049892223

cuadratic_model
--- Predictions ---
|2020-04-27 | 3939|
|2020-04-28 | 4078|
|2020-04-29 | 4218|
|2020-04-30 | 4360|
|2020-05-01 | 4505|
|2020-05-02 | 4651|
|2020-05-03 | 4799|
-------------------

 cuadratic_model
a  =  1.019595185126372
b  =  73.98098085189926
c  =  666.6857142871655

cubic_mo